## Preprocessing

In [14]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [15]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME', 'STATUS'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1


In [16]:
#Condensing the Preprocessing that was done in the first notebook into a function to make testing more efficient.
#Including variables to allow for changing the cutoff point for the 'Other' bins

def preprocess_df(df ,app_cut, class_cut):
    application_types_to_replace = []

    for value in df['APPLICATION_TYPE'].unique():
        if df['APPLICATION_TYPE'].value_counts()[value] < app_cut:
            application_types_to_replace.append(value)

    # Replace in dataframe
    for app in application_types_to_replace:
        df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

    classifications_to_replace = []

    for value in df['CLASSIFICATION'].unique():
        if df['CLASSIFICATION'].value_counts()[value] < class_cut:
            classifications_to_replace.append(value)

    # Replace in dataframe
    for cls in classifications_to_replace:
        df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,'Other')
        
    application_data = pd.get_dummies(df)
    
    # Split our preprocessed data into our features and target arrays
    X = application_data.drop(columns=['IS_SUCCESSFUL'])
    y = application_data['IS_SUCCESSFUL']

    # Split the preprocessed data into a training and testing dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)
    
    # Create a StandardScaler instances
    scaler = StandardScaler()

    # Fit the StandardScaler
    X_scaler = scaler.fit(X_train)

    # Scale the data
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, y_train, y_test

# Optimization Trials

## Bin Cutoffs -Trial 1

In [17]:
X_train_scaled, X_test_scaled, y_train, y_test = preprocess_df(application_df, 100, 100)

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  16
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 4

nn = tf.keras.models.Sequential()

# Hidden layers
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4, input_dim=number_input_features, activation="tanh")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))
# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 16)                800       
                                                                 
 dense_6 (Dense)             (None, 12)                204       
                                                                 
 dense_7 (Dense)             (None, 8)                 104       
                                                                 
 dense_8 (Dense)             (None, 4)                 36        
                                                                 
 dense_9 (Dense)             (None, 1)                 5         
                                                                 
Total params: 1,149
Trainable params: 1,149
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 1.3669 - accuracy: 0.6681
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6734 - accuracy: 0.7152
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5662 - accuracy: 0.7239
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5767 - accuracy: 0.7206
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5620 - accuracy: 0.7264
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5552 - accuracy: 0.7278
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5541 - accuracy: 0.7288
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5518 - accuracy: 0.7303
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5493 - accuracy: 0.7319
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5369 - accuracy: 0.7399
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5372 - accuracy: 0.7390
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5376 - accuracy: 0.7389
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5373 - accuracy: 0.7395
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5362 - accuracy: 0.7394
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5463 - accuracy: 0.7369
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5412 - accuracy: 0.7395
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5431 - accuracy: 0.7384
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5423 - accuracy: 0.7388
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5380 - accuracy: 

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5616 - accuracy: 0.7270 - 281ms/epoch - 1ms/step
Loss: 0.5615675449371338, Accuracy: 0.7269970774650574


Trial 1 Summary <br>
Applications Cutoff: 100 <br>
Classification Cutoff: 100 <br>
Nodes per Layer: 16, 12, 8, 4 <br>
Layer Activations: tanh, tanh, tanh, tanh <br>
Output Activation: relu <br>
Accuracy: 0.7269970774650574 <br>

## Trial 2

In [22]:
X_train_scaled, X_test_scaled, y_train, y_test = preprocess_df(application_df, 150, 100)

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  16
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 4

nn = tf.keras.models.Sequential()

# Hidden layers
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4, input_dim=number_input_features, activation="tanh")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))
# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 16)                800       
                                                                 
 dense_11 (Dense)            (None, 12)                204       
                                                                 
 dense_12 (Dense)            (None, 8)                 104       
                                                                 
 dense_13 (Dense)            (None, 4)                 36        
                                                                 
 dense_14 (Dense)            (None, 1)                 5         
                                                                 
Total params: 1,149
Trainable params: 1,149
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 1.4366 - accuracy: 0.6351
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.8913 - accuracy: 0.6958
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5793 - accuracy: 0.7208
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5649 - accuracy: 0.7252
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5599 - accuracy: 0.7248
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5568 - accuracy: 0.7276
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5553 - accuracy: 0.7284
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5782 - accuracy: 0.7169
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5536 - accuracy: 0.7292
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5509 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5375 - accuracy: 0.7396
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5368 - accuracy: 0.7383
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5369 - accuracy: 0.7394
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5371 - accuracy: 0.7393
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5367 - accuracy: 0.7399
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5372 - accuracy: 0.7394
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5362 - accuracy: 0.7405
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5371 - accuracy: 0.7391
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5382 - accuracy: 0.7401
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5403 - accuracy: 

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5518 - accuracy: 0.7262 - 284ms/epoch - 1ms/step
Loss: 0.5518245100975037, Accuracy: 0.7261807322502136


Trial 2 Summary <br>
Applications Cutoff: 150 <br>
Classification Cutoff: 100 <br>
Nodes per Layer: 16, 12, 8, 4 <br>
Layer Activations: tanh, tanh, tanh, tanh <br>
Output Activation: relu <br>
Accuracy: 0.7261807322502136 <br>

## Trial 3

In [27]:
X_train_scaled, X_test_scaled, y_train, y_test = preprocess_df(application_df, 200, 100)

In [28]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  16
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 4

nn = tf.keras.models.Sequential()

# Hidden layers
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4, input_dim=number_input_features, activation="tanh")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))
# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 16)                784       
                                                                 
 dense_16 (Dense)            (None, 12)                204       
                                                                 
 dense_17 (Dense)            (None, 8)                 104       
                                                                 
 dense_18 (Dense)            (None, 4)                 36        
                                                                 
 dense_19 (Dense)            (None, 1)                 5         
                                                                 
Total params: 1,133
Trainable params: 1,133
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 1.6264 - accuracy: 0.6768
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.7076 - accuracy: 0.7153
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6833 - accuracy: 0.7049
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6039 - accuracy: 0.7172
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5915 - accuracy: 0.7217
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5879 - accuracy: 0.7242
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5612 - accuracy: 0.7234
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5539 - accuracy: 0.7287
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5842 - accuracy: 0.7031
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6008 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5384 - accuracy: 0.7369
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5396 - accuracy: 0.7368
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5383 - accuracy: 0.7391
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5389 - accuracy: 0.7394
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5381 - accuracy: 0.7378
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5412 - accuracy: 0.7383
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5379 - accuracy: 0.7379
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5378 - accuracy: 0.7390
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5377 - accuracy: 0.7376
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5380 - accuracy: 

In [31]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5542 - accuracy: 0.7275 - 297ms/epoch - 1ms/step
Loss: 0.5542187094688416, Accuracy: 0.7274635434150696


Trial 3 Summary <br>
Applications Cutoff: 200 <br>
Classification Cutoff: 100 <br>
Nodes per Layer: 16, 12, 8, 4 <br>
Layer Activations: tanh, tanh, tanh, tanh <br>
Output Activation: relu <br>
Accuracy: 0.7274635434150696 <br>

## Trial 4

In [32]:
X_train_scaled, X_test_scaled, y_train, y_test = preprocess_df(application_df, 100, 150)

In [33]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  16
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 4

nn = tf.keras.models.Sequential()

# Hidden layers
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4, input_dim=number_input_features, activation="tanh")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))
# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 16)                736       
                                                                 
 dense_21 (Dense)            (None, 12)                204       
                                                                 
 dense_22 (Dense)            (None, 8)                 104       
                                                                 
 dense_23 (Dense)            (None, 4)                 36        
                                                                 
 dense_24 (Dense)            (None, 1)                 5         
                                                                 
Total params: 1,085
Trainable params: 1,085
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 2.5101 - accuracy: 0.5882
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.9468 - accuracy: 0.6817
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5798 - accuracy: 0.7173
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.7216 - accuracy: 0.6866
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6204 - accuracy: 0.6978
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5826 - accuracy: 0.7104
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5703 - accuracy: 0.7175
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5648 - accuracy: 0.7211
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5659 - accuracy: 0.7203
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5612 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5395 - accuracy: 0.7381
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5392 - accuracy: 0.7384
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5396 - accuracy: 0.7386
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5396 - accuracy: 0.7383
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5391 - accuracy: 0.7387
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5388 - accuracy: 0.7384
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5395 - accuracy: 0.7383
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5397 - accuracy: 0.7378
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5389 - accuracy: 0.7392
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5395 - accuracy: 

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5598 - accuracy: 0.7277 - 280ms/epoch - 1ms/step
Loss: 0.5597615242004395, Accuracy: 0.7276967763900757


Trial 4 Summary <br>
Applications Cutoff: 100 <br>
Classification Cutoff: 150 <br>
Nodes per Layer: 16, 12, 8, 4 <br>
Layer Activations: tanh, tanh, tanh, tanh <br>
Output Activation: relu <br>
Accuracy: 0.7276967763900757 <br>

## Trial 5

In [37]:
X_train_scaled, X_test_scaled, y_train, y_test = preprocess_df(application_df, 100, 200)

In [38]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  16
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 4

nn = tf.keras.models.Sequential()

# Hidden layers
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4, input_dim=number_input_features, activation="tanh")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))
# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 16)                720       
                                                                 
 dense_26 (Dense)            (None, 12)                204       
                                                                 
 dense_27 (Dense)            (None, 8)                 104       
                                                                 
 dense_28 (Dense)            (None, 4)                 36        
                                                                 
 dense_29 (Dense)            (None, 1)                 5         
                                                                 
Total params: 1,069
Trainable params: 1,069
Non-trainable params: 0
_________________________________________________________________


In [39]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [40]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 1.4522 - accuracy: 0.6637
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5987 - accuracy: 0.6988
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5718 - accuracy: 0.7177
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5688 - accuracy: 0.7227
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5577 - accuracy: 0.7261
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5687 - accuracy: 0.7157
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5652 - accuracy: 0.7228
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5563 - accuracy: 0.7285
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5551 - accuracy: 0.7292
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5533 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5386 - accuracy: 0.7395
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5393 - accuracy: 0.7393
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5388 - accuracy: 0.7385
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5386 - accuracy: 0.7379
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5398 - accuracy: 0.7393
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5398 - accuracy: 0.7388
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5389 - accuracy: 0.7386
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5383 - accuracy: 0.7385
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5388 - accuracy: 0.7386
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5383 - accuracy: 

In [41]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5601 - accuracy: 0.7263 - 323ms/epoch - 1ms/step
Loss: 0.5601485371589661, Accuracy: 0.7262973785400391


Trial 5 Summary <br>
Applications Cutoff: 100 <br>
Classification Cutoff: 200 <br>
Nodes per Layer: 16, 12, 8, 4 <br>
Layer Activations: tanh, tanh, tanh, tanh <br>
Output Activation: relu <br>
Accuracy: 0.7262973785400391 <br>

## Trial 6

In [42]:
X_train_scaled, X_test_scaled, y_train, y_test = preprocess_df(application_df, 150, 150)

In [43]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  16
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 4

nn = tf.keras.models.Sequential()

# Hidden layers
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4, input_dim=number_input_features, activation="tanh")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))
# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 16)                720       
                                                                 
 dense_31 (Dense)            (None, 12)                204       
                                                                 
 dense_32 (Dense)            (None, 8)                 104       
                                                                 
 dense_33 (Dense)            (None, 4)                 36        
                                                                 
 dense_34 (Dense)            (None, 1)                 5         
                                                                 
Total params: 1,069
Trainable params: 1,069
Non-trainable params: 0
_________________________________________________________________


In [44]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [45]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 2.0001 - accuracy: 0.6105
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.8847 - accuracy: 0.6798
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6835 - accuracy: 0.6993
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6311 - accuracy: 0.7122
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6039 - accuracy: 0.7121
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5776 - accuracy: 0.7187
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5697 - accuracy: 0.7220
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5674 - accuracy: 0.7242
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5614 - accuracy: 0.7262
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5679 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5416 - accuracy: 0.7379
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5398 - accuracy: 0.7386
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5448 - accuracy: 0.7369
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5406 - accuracy: 0.7377
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5396 - accuracy: 0.7385
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5398 - accuracy: 0.7379
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5394 - accuracy: 0.7381
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5395 - accuracy: 0.7382
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5395 - accuracy: 0.7392
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5395 - accuracy: 

In [46]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5562 - accuracy: 0.7268 - 296ms/epoch - 1ms/step
Loss: 0.5561833381652832, Accuracy: 0.7267638444900513


Trial 6 Summary <br>
Applications Cutoff: 150 <br>
Classification Cutoff: 150 <br>
Nodes per Layer: 16, 12, 8, 4 <br>
Layer Activations: tanh, tanh, tanh, tanh <br>
Output Activation: relu <br>
Accuracy: 0.7267638444900513 <br>

## Trial 7

In [47]:
X_train_scaled, X_test_scaled, y_train, y_test = preprocess_df(application_df, 200, 200)

In [48]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  16
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 4

nn = tf.keras.models.Sequential()

# Hidden layers
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4, input_dim=number_input_features, activation="tanh")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))
# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_35 (Dense)            (None, 16)                720       
                                                                 
 dense_36 (Dense)            (None, 12)                204       
                                                                 
 dense_37 (Dense)            (None, 8)                 104       
                                                                 
 dense_38 (Dense)            (None, 4)                 36        
                                                                 
 dense_39 (Dense)            (None, 1)                 5         
                                                                 
Total params: 1,069
Trainable params: 1,069
Non-trainable params: 0
_________________________________________________________________


In [49]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [50]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 1.1533 - accuracy: 0.6644
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6651 - accuracy: 0.7070
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6460 - accuracy: 0.7218
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5791 - accuracy: 0.7206
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5659 - accuracy: 0.7282
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5575 - accuracy: 0.7294
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5532 - accuracy: 0.7289
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5604 - accuracy: 0.7272
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5532 - accuracy: 0.7322
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5543 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5412 - accuracy: 0.7374
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5395 - accuracy: 0.7380
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5389 - accuracy: 0.7384
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5391 - accuracy: 0.7375
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5395 - accuracy: 0.7378
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5389 - accuracy: 0.7389
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5397 - accuracy: 0.7378
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5390 - accuracy: 0.7390
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5390 - accuracy: 0.7379
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5390 - accuracy: 

In [51]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5593 - accuracy: 0.7266 - 283ms/epoch - 1ms/step
Loss: 0.5592740774154663, Accuracy: 0.7266472578048706


Trial 7 Summary <br>
Applications Cutoff: 150 <br>
Classification Cutoff: 150 <br>
Nodes per Layer: 16, 12, 8, 4 <br>
Layer Activations: tanh, tanh, tanh, tanh <br>
Output Activation: relu <br>
Accuracy: 0.7266472578048706 <br>

# Model Trials
Bin Cutoff values from Trial 4 (100, 150) yielded the best accuracy (0.7276967763900757) so they will be used in subsequent tests.

In [88]:
X_train_scaled, X_test_scaled, y_train, y_test = preprocess_df(application_df, 100, 150)

## Testing tanh and relu alternating

In [93]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  16
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 4

nn = tf.keras.models.Sequential()

# Hidden layers
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="relu")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4, input_dim=number_input_features, activation="relu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))
# Check the structure of the model
nn.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_82 (Dense)            (None, 16)                720       
                                                                 
 dense_83 (Dense)            (None, 12)                204       
                                                                 
 dense_84 (Dense)            (None, 8)                 104       
                                                                 
 dense_85 (Dense)            (None, 4)                 36        
                                                                 
 dense_86 (Dense)            (None, 1)                 5         
                                                                 
Total params: 1,069
Trainable params: 1,069
Non-trainable params: 0
_________________________________________________________________


In [94]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [95]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.8357 - accuracy: 0.6502
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6056 - accuracy: 0.7182
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5778 - accuracy: 0.7249
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5747 - accuracy: 0.7269
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5697 - accuracy: 0.7281
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5670 - accuracy: 0.7293
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5657 - accuracy: 0.7302
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5630 - accuracy: 0.7285
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5560 - accuracy: 0.7303
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5600 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5400 - accuracy: 0.7385
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5396 - accuracy: 0.7372
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5398 - accuracy: 0.7383
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5396 - accuracy: 0.7385
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5406 - accuracy: 0.7380
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5396 - accuracy: 0.7378
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5399 - accuracy: 0.7378
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5400 - accuracy: 0.7379
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5396 - accuracy: 0.7390
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5394 - accuracy: 

In [96]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5651 - accuracy: 0.7275 - 278ms/epoch - 1ms/step
Loss: 0.5650548934936523, Accuracy: 0.7274635434150696


Trial Summary <br>
Applications Cutoff: 100 <br>
Classification Cutoff: 150 <br>
Nodes per Layer: 16, 12, 8, 4 <br>
Layer Activations: tanh, relu, tanh, relu <br>
Output Activation: relu <br>
Accuracy: 0.7265306115150452 <br>
Lower Accuracy

## Testing tanh and relu alternating, sig output

In [97]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  16
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 4

nn = tf.keras.models.Sequential()

# Hidden layers
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="relu")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4, input_dim=number_input_features, activation="relu")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_87 (Dense)            (None, 16)                720       
                                                                 
 dense_88 (Dense)            (None, 12)                204       
                                                                 
 dense_89 (Dense)            (None, 8)                 104       
                                                                 
 dense_90 (Dense)            (None, 4)                 36        
                                                                 
 dense_91 (Dense)            (None, 1)                 5         
                                                                 
Total params: 1,069
Trainable params: 1,069
Non-trainable params: 0
_________________________________________________________________


In [98]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [99]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5941 - accuracy: 0.7037
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5616 - accuracy: 0.7277
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5554 - accuracy: 0.7318
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5523 - accuracy: 0.7316
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5510 - accuracy: 0.7313
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5496 - accuracy: 0.7327
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5484 - accuracy: 0.7324
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5480 - accuracy: 0.7324
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5473 - accuracy: 0.7332
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5379 - accuracy: 0.7386
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5381 - accuracy: 0.7385
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5380 - accuracy: 0.7380
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5379 - accuracy: 0.7384
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5377 - accuracy: 0.7382
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5381 - accuracy: 0.7385
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5378 - accuracy: 0.7378
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5380 - accuracy: 0.7383
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5375 - accuracy: 0.7392
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5376 - accuracy: 

In [100]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5537 - accuracy: 0.7269 - 346ms/epoch - 1ms/step
Loss: 0.5537497997283936, Accuracy: 0.7268804907798767


Trial Summary <br>
Applications Cutoff: 100 <br>
Classification Cutoff: 150 <br>
Nodes per Layer: 16, 12, 8, 4 <br>
Layer Activations: tanh, relu, tanh, relu <br>
Output Activation: sig <br>
Accuracy: 0.7268804907798767 <br>
Lower Accuracy

## Testing sigmoid activation on output

In [57]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  16
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 4

nn = tf.keras.models.Sequential()

# Hidden layers
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4, input_dim=number_input_features, activation="tanh")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 16)                720       
                                                                 
 dense_46 (Dense)            (None, 12)                204       
                                                                 
 dense_47 (Dense)            (None, 8)                 104       
                                                                 
 dense_48 (Dense)            (None, 4)                 36        
                                                                 
 dense_49 (Dense)            (None, 1)                 5         
                                                                 
Total params: 1,069
Trainable params: 1,069
Non-trainable params: 0
_________________________________________________________________


In [58]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [59]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5880 - accuracy: 0.7124
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5607 - accuracy: 0.7301
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5554 - accuracy: 0.7322
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5535 - accuracy: 0.7328
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5514 - accuracy: 0.7336
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5502 - accuracy: 0.7330
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5492 - accuracy: 0.7326
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5487 - accuracy: 0.7339
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5479 - accuracy: 0.7334
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5472 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5378 - accuracy: 0.7380
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5375 - accuracy: 0.7392
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5380 - accuracy: 0.7390
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5375 - accuracy: 0.7384
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5376 - accuracy: 0.7386
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5373 - accuracy: 0.7394
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5373 - accuracy: 0.7390
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5373 - accuracy: 0.7385
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5376 - accuracy: 0.7391
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5373 - accuracy: 

In [60]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5537 - accuracy: 0.7280 - 273ms/epoch - 1ms/step
Loss: 0.5537335872650146, Accuracy: 0.7280466556549072


Trial Summary <br>
Applications Cutoff: 100 <br>
Classification Cutoff: 150 <br>
Nodes per Layer: 16, 12, 8, 4 <br>
Layer Activations: tanh, tanh, tanh, tanh <br>
Output Activation: sig <br>
Accuracy: 0.7280466556549072 <br>
Higher Accuracy

## Testing 3 hidden layers, no other changes

In [61]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  16
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 8

nn = tf.keras.models.Sequential()

# Hidden layers
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, input_dim=number_input_features, activation="tanh")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))
# Check the structure of the model
nn.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_50 (Dense)            (None, 16)                720       
                                                                 
 dense_51 (Dense)            (None, 12)                204       
                                                                 
 dense_52 (Dense)            (None, 8)                 104       
                                                                 
 dense_53 (Dense)            (None, 1)                 9         
                                                                 
Total params: 1,037
Trainable params: 1,037
Non-trainable params: 0
_________________________________________________________________


In [62]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [63]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 2.2252 - accuracy: 0.6028
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 1.2971 - accuracy: 0.6786
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.9330 - accuracy: 0.7020
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.7296 - accuracy: 0.7170
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.7113 - accuracy: 0.7245
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.7087 - accuracy: 0.7253
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6205 - accuracy: 0.7231
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5841 - accuracy: 0.7263
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5736 - accuracy: 0.7287
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6463 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.7373
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5432 - accuracy: 0.7372
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5426 - accuracy: 0.7367
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5461 - accuracy: 0.7369
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5473 - accuracy: 0.7352
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5419 - accuracy: 0.7366
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5411 - accuracy: 0.7385
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5431 - accuracy: 0.7374
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5499 - accuracy: 0.7356
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5441 - accuracy: 

In [64]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5570 - accuracy: 0.7276 - 289ms/epoch - 1ms/step
Loss: 0.5569878220558167, Accuracy: 0.727580189704895


Trial Summary <br>
Applications Cutoff: 100 <br>
Classification Cutoff: 150 <br>
Nodes per Layer: 16, 12, 8 <br>
Layer Activations: tanh, tanh, tanh <br>
Output Activation: relu <br>
Accuracy: 0.7269970774650574 <br>
Lower Accuracy

# Saving the Most Optimized Version
The test shows that the sigmoid activation on the output layer is slightly more optimal than the initial model.

In [78]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  16
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 8
hidden_nodes_layer4 = 4

nn = tf.keras.models.Sequential()

# Hidden layers
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer3, input_dim=number_input_features, activation="tanh")
)

nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer4, input_dim=number_input_features, activation="tanh")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_67 (Dense)            (None, 16)                720       
                                                                 
 dense_68 (Dense)            (None, 12)                204       
                                                                 
 dense_69 (Dense)            (None, 8)                 104       
                                                                 
 dense_70 (Dense)            (None, 4)                 36        
                                                                 
 dense_71 (Dense)            (None, 1)                 5         
                                                                 
Total params: 1,069
Trainable params: 1,069
Non-trainable params: 0
_________________________________________________________________


In [79]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [80]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5802 - accuracy: 0.7146
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5585 - accuracy: 0.7283
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5541 - accuracy: 0.7314
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5513 - accuracy: 0.7309
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5498 - accuracy: 0.7318
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5486 - accuracy: 0.7327
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5476 - accuracy: 0.7338
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5466 - accuracy: 0.7327
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5463 - accuracy: 0.7350
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5458 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5367 - accuracy: 0.7392
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5367 - accuracy: 0.7390
Epoch 83/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5365 - accuracy: 0.7404
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5364 - accuracy: 0.7398
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5364 - accuracy: 0.7396
Epoch 86/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5365 - accuracy: 0.7399
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5364 - accuracy: 0.7397
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5366 - accuracy: 0.7398
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5362 - accuracy: 0.7395
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5363 - accuracy: 

In [81]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5525 - accuracy: 0.7265 - 289ms/epoch - 1ms/step
Loss: 0.5524861812591553, Accuracy: 0.7265306115150452


In [82]:
# Export our model to HDF5 file
nn.save_weights('AlphabetSoupCharity_Optimization.h5')